In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## DATA PREPARATION

In [ ]:
!pip install html-table-parser-python3 --quiet

In [ ]:
import urllib.request
from html_table_parser.parser import HTMLTableParser
import pandas as pd

In [ ]:
def url_get_contents(url):
    
    hdrs = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    req = urllib.request.Request(url=url, headers=hdrs)
    f = urllib.request.urlopen(req)

    return f.read()

## testing data

In [ ]:
link = 'https://www.goal.com/en/news/arsenal-premier-league-fixtures-full-2021-22-schedule/uj7gk3k386a3zl6yyubv9kzy'
xhtml = url_get_contents(link).decode('utf-8')
p = HTMLTableParser()
p.feed(xhtml)
len(p.tables)
m = p.tables[0]
df_table = pd.DataFrame(m)
df_table.columns = df_table.iloc[0,:]
df_table = df_table.iloc[1:,]

In [ ]:
df_table['opponent'] = df_table['Match'].apply(lambda x: x.replace(' v ', ''))
df_table['opponent'] = df_table['opponent'].apply(lambda x: x.lower().replace('arsenal',''))
df_table['opponent'] = df_table['opponent'].apply(lambda x: x.lower().replace(' ',''))

In [ ]:
df_table['home'] = df_table['Match'].apply(lambda x: x.split(' v ')[0].lower())
df_table['away'] = df_table['Match'].apply(lambda x: x.split(' v ')[1].lower())
def home_adv(x, team):
    if x == team:
        return 1
    else:
        return 0
df_table['home-adv'] = 0
df_table['home-adv'] = df_table['home'].apply(lambda x: home_adv(x, 'arsenal'))

In [ ]:
df_table.to_csv('arsenal_season-2021-22.csv', index=None)

## Training data

In [ ]:
# Scrape for all seasons as you like and append them, I have done only for the 18/19 - 20/21 season
# Will get the points table and ratio of points for each team'
link = "https://en.wikipedia.org/wiki/2020%E2%80%9321_Premier_League"
xhtml = url_get_contents(link).decode('utf-8')
p = HTMLTableParser()
p.feed(xhtml)
m = p.tables[-18]
df_table = pd.DataFrame(m)
df_table.columns = df_table.iloc[0,:]
df_table = df_table.iloc[1:,]
df_table.columns = ['Pos',
       'Team',
       'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts',
       'Qualification or relegation']

In [ ]:
df_table['Pts'] = [int(x) for x in df_table['Pts']] 
df_table['PI'] = [x/df_table['Pts'].median() for x in df_table['Pts']]
df_mod = df_table[['Team', 'Pts', 'PI']]

In [ ]:
# df_all = pd.DataFrame()
df_all = df_all.append(df_mod)

In [ ]:
df_all['Team'] = df_all['Team'].apply(lambda x: x.replace("(C)", ''))
df_all['Team'] = df_all['Team'].apply(lambda x: x.replace("(R)", ''))

In [ ]:
df_all

In [ ]:
df_all_mod = df_all.groupby(['Team']).mean()
df_all_mod = df_all_mod.sort_values(by=['PI'], ascending=False)
df_all_mod['Team'] = df_all_mod.index
df_all_mod.index = list(range(0, df_all_mod.shape[0]))
df_all_mod['Team'] = df_all_mod['Team'].apply(lambda x: x.lower())

In [ ]:
df_all_mod['Team'] = df_all_mod['Team'].apply(lambda x: x.replace(" ", ""))

In [ ]:
df_all_mod.to_csv('working/PI_clubs_epl.csv')

In [ ]:
df_ars = pd.read_csv('input/arsenal-epl-2019-2021/prem-arsenal.csv')
df_ars['opponent'] = df_ars['home'] + df_ars['away']
df_ars['opponent'] = df_ars['opponent'].apply(lambda x: x.replace('arsenal', ''))
df_ars['opponent'] = df_ars['opponent'].apply(lambda x: x.replace(' ', ''))

In [ ]:
dict_prem = {}
for i in range(0, df_all_mod.shape[0]):
    dict_prem[df_all_mod['Team'].iloc[i]] = df_all_mod['PI'].iloc[i]
    
df_ars['PI'] = [dict_prem[x] for x in df_ars['opponent']]

In [ ]:
ars_pi = df_all_mod['PI'][df_all_mod['Team'] == 'arsenal']
df_ars['PI'] = df_ars['PI']/ars_pi.values

In [ ]:
df_ars.to_csv('working/prem_arsenal.csv', index=None)